In [1]:
import argparse
import errno
import glob
import json
import os
import re
import shutil
from types import SimpleNamespace
import cv2
import torch
import warnings
from lightning_objects import LightningModel
warnings.filterwarnings('ignore')
from config import Configuration
import pandas as pd
from utils import stratify_split, make_holdout_df, set_seeds
from train_manager import TrainManager

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def main(experiment_name: str, debug, resume=False,
         finetune=False, freeze_bn=True, freeze_feature_extractor=False,
         data_csv='/train.csv'):

    experiment_dir = os.path.abspath(f'trained-models/{experiment_name}')
    print('Experiment directory', experiment_dir)

    try:
        # -------- SETUP --------
        checkpoint_params = None
        finetune_model_fnames = None
        folds_df, holdout_df = None, None

        if not resume and not finetune: # totally new experiment
            make_experiment_directory(experiment_dir)
            config = Configuration()
            config.debug = debug
            set_seeds(config.seed)

            # -------- LOAD DATA FROM TRAIN FILE --------
            data_df = pd.read_csv(config.data_dir + data_csv, engine='python')
            data_df, holdout_df = make_holdout_df(data_df, seed=config.seed)
            folds_df = stratify_split(data_df, config.fold_num, config.seed, config.target_col)

            # -------- SAVE FILES (experiment state: things like resuming, fine tuning, and inference on holdout) --------
            folds_df.to_csv(experiment_dir + '/folds.csv', index=False)
            holdout_df.to_csv(experiment_dir + '/holdout.csv', index=False)
            with open(experiment_dir + '/experiment_config.json', 'w') as f:
                json.dump(config.__dict__, f)
        elif resume or finetune:
            # LOAD DATA FROM SAVED FILES
            with open(experiment_dir + '/experiment_config.json', 'r') as f:
                config = json.load(f, object_hook=lambda d: SimpleNamespace(**d))
                set_seeds(config.seed)
                config.debug = debug
            
            holdout_df = pd.read_csv(experiment_dir + '/holdout.csv', engine='python')
            if data_csv != '/train.csv': # create new folds from new data file
                data_df = pd.read_csv(config.data_dir + data_csv, engine='python') 
                data_df, _ = make_holdout_df(data_df, seed=config.seed) # use same previous holdout with noise
                folds_df = stratify_split(data_df, config.fold_num, config.seed, config.target_col)
            else: # use existing folds
                folds_df = pd.read_csv(experiment_dir + '/folds.csv', engine='python')

            if finetune and not resume:
                print('finetuning...')
                # verify there are checkpoints to fine tune
                finetune_model_fnames = glob.glob(experiment_dir + '/*fold*.ckpt')
                assert len(finetune_model_fnames) > 0
                finetune_model_fnames.sort()

                # make new directory for tuning experiment with files from training run 1
                make_experiment_directory(experiment_dir + '_tune')
                for f in os.listdir(experiment_dir):
                    print(f"copying {f} to {experiment_dir + '_tune'}")
                    shutil.copy2(experiment_dir + '/' + f, experiment_dir + '_tune')
                experiment_dir += '_tune'
                experiment_name += '_tune'
    
                # overwrite folds from old experiment
                folds_df.to_csv(experiment_dir + '/folds.csv', index=False) 
                
            else:
                print('resuming from last checkpoint...')
                checkpoint_params = get_checkpoint_params(experiment_dir, resume)

        assert holdout_df is not None, 'holdout_df is None'
        assert folds_df is not None, 'folds_df is None'

        # cv2 multithreading seems to go into deadlock with PyTorch data loaders
        if config.num_workers > 0:
            cv2.setNumThreads(0)

        config.lr = 0.00003
        config.lr_test = True
        config.train_bs = 8

        trainer = TrainManager(experiment_name=experiment_name, experiment_dir=experiment_dir,
                               folds_df=folds_df, holdout_df=holdout_df,
                               checkpoint_params=checkpoint_params, config=config,
                               finetune=finetune, freeze_bn=freeze_bn,
                               freeze_feature_extractor=freeze_feature_extractor,
                               finetune_model_fnames=finetune_model_fnames)
        trainer.run()
    finally:
        torch.cuda.empty_cache()

def make_experiment_directory(name):
    try:
        os.makedirs(name)
    except FileExistsError as e:
        print('Experiment already exists. Be sure to resume training appropriately or start a new experiment.')
        if e.errno == errno.EEXIST: raise


def get_checkpoint_params(basename, resume):
    checkpoint_params = None
    if resume:
        checkpoint_params = {}
        model_filenames = glob.glob(basename + '/*fold*.ckpt')
        model_filenames.sort()
        trained_folds = [re.findall(r'fold\d+', f)[0][len('fold'):] for f in model_filenames]
        most_recent_fold = int(max(trained_folds)) if len(trained_folds) > 0 else 0

        checkpoint_params['restart_from'] = most_recent_fold
        checkpoint_params['checkpoint_file_path'] = model_filenames[-1]

    return checkpoint_params

In [4]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Kaggle Cassava Disease Training')

    try:
        debug = False
        print('Running in debug mode:', debug)
        # fine tuned with SGD, one cycle
        main(experiment_name='sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53', debug=debug,
             resume=False, finetune=True, freeze_bn=True, freeze_feature_extractor=False,
            data_csv='/train_cleaned-0.6.csv')
    except KeyboardInterrupt:
        pass

Running in debug mode: False
Experiment directory /opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53
finetuning...
copying tf_efficientnet_b4_ns_bitempered_smooth=0.05_val_loss=0.103_val_acc=0.889_fold1.ckpt to /opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53_tune
copying tf_efficientnet_b4_ns_bitempered_smooth=0.05_val_loss=0.109_val_acc=0.878_fold2.ckpt to /opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53_tune
copying experiment_config.json to /opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53_tune
copying tf_efficientnet_b4_ns_bitempered_smooth=0.05_val_loss=0.104_val_acc=0.887_fold3.ckpt to /opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53_tune
copying folds.csv to /opt/fav

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Models to fine tune
 ['/opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53/tf_efficientnet_b4_ns_bitempered_smooth=0.05_val_loss=0.101_val_acc=0.892_fold0.ckpt', '/opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53/tf_efficientnet_b4_ns_bitempered_smooth=0.05_val_loss=0.103_val_acc=0.889_fold1.ckpt', '/opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53/tf_efficientnet_b4_ns_bitempered_smooth=0.05_val_loss=0.104_val_acc=0.887_fold3.ckpt', '/opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53/tf_efficientnet_b4_ns_bitempered_smooth=0.05_val_loss=0.108_val_acc=0.883_fold4.ckpt', '/opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53/tf_efficientnet_b4_ns_bitempered_smooth=0.05_val_loss=0.109_


  | Name           | Type           | Params
--------------------------------------------------
0 | valid_accuracy | Accuracy       | 0     
1 | test_accuracy  | Accuracy       | 0     
2 | criterion      | BiTemperedLoss | 0     
3 | model          | EfficientNet   | 17.6 M
--------------------------------------------------
17.4 M    Trainable params
125 K     Non-trainable params
17.6 M    Total params
Finding best initial lr: 100%|██████████| 100/100 [01:37<00:00,  1.01it/s]Restored states from the checkpoint file at /opt/favordata/AI/Felix/kaggle-cassava/trained-models/sgd_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-53_tune/lr_find_temp_model.ckpt
Learning rate set to 1e-06

  | Name           | Type           | Params
--------------------------------------------------
0 | valid_accuracy | Accuracy       | 0     
1 | test_accuracy  | Accuracy       | 0     
2 | criterion      | BiTemperedLoss | 0     
3 | model          | EfficientNet   | 17.6 M
---------------------------

Epoch 1:   9%|▉         | 167/1770 [00:42<-1:55:08, -5.48it/s, loss=0.168, v_num=7, val_loss=0.0557, val_acc=0.961, train_loss=0.0484] 

...

Epoch 13: 100%|██████████| 1770/1770 [07:43<00:00,  2.96it/s, loss=0.186, v_num=7, val_loss=0.0743, val_acc=0.941, train_loss=0.0467]
                                                           

Trainer was signaled to stop but required minimum epochs (15) or minimum steps (None) has not been met. Training will continue...


Epoch 14:   0%|          | 2/1770 [00:00<-1:59:58, -602.12it/s, loss=0.186, v_num=7, val_loss=0.0743, val_acc=0.941, train_loss=0.0467]  
Validating: 0it [00:00, ?it/s]
Epoch 14:   3%|▎         | 56/1770 [00:33<-1:57:15, -10.36it/s, loss=0.186, v_num=7, val_loss=0.0743, val_acc=0.941, train_loss=0.0467]

Epoch 14, step 5578: val_loss was not in top 1


Epoch 14:   3%|▎         | 56/1770 [00:33<-1:57:15, -10.33it/s, loss=0.186, v_num=7, val_loss=0.0736, val_acc=0.943, train_loss=0.394] 
                                                           